The Nobel Prize offers a Web API https://nobelprize.readme.io/docs/prize

In [1]:
from pymongo import MongoClient
from pprint import pprint
import requests 
import json

In [3]:
# Client connects to "localhost" by default 
client = MongoClient()

In [4]:
# Create local "nobel" database on the fly 
db = client["nobel"]
db.prizes.drop()
db.laureates.drop()
# API documented at https://nobelprize.readme.io/docs/prize 
for collection_name in ["prizes", "laureates"]:
    singular = collection_name[:-1] # the API uses singular
    response = requests.get( "http://api.nobelprize.org/v1/{}.json".format(singular)) 
    documents = response.json()[collection_name] 
    # Create collections on the fly 
    db[collection_name].insert_many(documents)

In [5]:
pprint(db.laureates.find_one())

{'_id': ObjectId('648b6dcd5a4e4bc30463e583'),
 'born': '1845-03-27',
 'bornCity': 'Lennep (now Remscheid)',
 'bornCountry': 'Prussia (now Germany)',
 'bornCountryCode': 'DE',
 'died': '1923-02-10',
 'diedCity': 'Munich',
 'diedCountry': 'Germany',
 'diedCountryCode': 'DE',
 'firstname': 'Wilhelm Conrad',
 'gender': 'male',
 'id': '1',
 'prizes': [{'affiliations': [{'city': 'Munich',
                               'country': 'Germany',
                               'name': 'Munich University'}],
             'category': 'physics',
             'motivation': '"in recognition of the extraordinary services he '
                           'has rendered by the discovery of the remarkable '
                           'rays subsequently named after him"',
             'share': '1',
             'year': '1901'}],
 'surname': 'Röntgen'}


In [6]:
# TODO: Print the first name of the first document
first_document = db.laureates.find_one()
print(first_document["firstname"])

Wilhelm Conrad


In [7]:
# How many female laureates exists? 
female_laureates_count = db.laureates.count_documents({"gender": "female"})
print("Number of female laureates:", female_laureates_count)

Number of female laureates: 60


In [8]:
db.laureates.distinct("bornCountry", {"bornCountry": {"$regex": "Germany"}})

['Bavaria (now Germany)',
 'East Friesland (now Germany)',
 'Germany',
 'Germany (now France)',
 'Germany (now Poland)',
 'Germany (now Russia)',
 'Hesse-Kassel (now Germany)',
 'Mecklenburg (now Germany)',
 'Prussia (now Germany)',
 'Schleswig (now Germany)',
 'West Germany (now Germany)',
 'Württemberg (now Germany)']

In [9]:
# TODO: How many laureates are from Germany?
german_laureates_count = db.laureates.count_documents({"birth.country": "Germany"})
print("Number of laureates from Germany:", german_laureates_count)

Number of laureates from Germany: 0


In [33]:
# TODO: Find all physics nobel laureates that are from Germany
# print the year of the first prize, the first name, and surname

german_physics_laureates = db.laureates.find({"bornCountry": {"$regex": "Germany"}, "prizes.category": "physics"})
#for testing
count = db.laureates.count_documents({"bornCountry": {"$regex": "Germany"}, "prizes.category": "physics"})
print(count)

32


In [39]:
first_physics_laureate = db.laureates.find_one({"bornCountry": {"$regex": "Germany"}, "prizes.category": "physics"})

if first_physics_laureate:
    first_prize = first_physics_laureate['prizes'][0]
    year = first_prize.get('year')
    firstname = first_physics_laureate['firstname']
    surname = first_physics_laureate['surname']

    if year:
        print(year, firstname, surname)
    else:
        print("Year not available for the first physics Nobel laureate from Germany.")
else:
    print("No physics Nobel laureates from Germany found.")

1901 Wilhelm Conrad Röntgen


In [40]:
# TODO: find and print the document for "Malala" (firstname)
malala_document = db.laureates.find_one({"firstname": "Malala"})

if malala_document:
    print(malala_document)
else:
    print("No document found for Malala.")


{'_id': ObjectId('648b6dce5a4e4bc30463e8f3'), 'id': '914', 'firstname': 'Malala', 'surname': 'Yousafzai', 'born': '1997-07-12', 'died': '0000-00-00', 'bornCountry': 'Pakistan', 'bornCountryCode': 'PK', 'bornCity': 'Mingora', 'gender': 'female', 'prizes': [{'year': '2014', 'category': 'peace', 'share': '2', 'motivation': '"for their struggle against the suppression of children and young people and for the right of all children to education"', 'affiliations': [[]]}]}


# sort()

In [65]:
# TODO: Find only female nobel laureates 
# and sort them according the the prize year in ascending order
# print year of the first prize, firstname, and surename
female_laureates = db.laureates.find({"gender": "female", "prizes": {"$exists": True}}).sort("prizes.year", 1)
#for checkin
num_females = db.laureates.count_documents({"gender": "female", "prizes": {"$exists": True}})
print(num_females) #output 60

60


In [66]:
if num_females > 0:
    first_female_laureate = female_laureates[0]
    first_prize = first_female_laureate['prizes'][0]
    year = first_prize['year']
    firstname = first_female_laureate['firstname']
    surname = first_female_laureate['surname']

    print(year, firstname, surname)
else:
    print("No female Nobel laureates found.")

1903 Marie Curie
